<a href="https://colab.research.google.com/github/willianpsantos/gpu-processing/blob/main/gpu_formula_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 553, done.
remote: Counting objects: 100% (284/284), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 553 (delta 179), reused 147 (delta 100), pack-reused 269 (from 1)
Receiving objects: 100% (553/553), 178.44 KiB | 14.87 MiB/s, done.
Resolving deltas: 100% (281/281), done.


In [3]:
!pip3 install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [4]:
!python3 /content/rapidsai-csp-utils/colab/pip-install.py

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pynvml.py", line 2248, in _LoadNvmlLibrary
    nvmlLib = CDLL("libnvidia-ml.so.1")
  File "/usr/lib/python3.10/ctypes/__init__.py", line 374, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: libnvidia-ml.so.1: cannot open shared object file: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 18, in <module>
    pynvml.nvmlInit()
  File "/usr/local/lib/python3.10/dist-packages/pynvml.py", line 2220, in nvmlInit
    nvmlInitWithFlags(0)
  File "/usr/local/lib/python3.10/dist-packages/pynvml.py", line 2203, in nvmlInitWithFlags
    _LoadNvmlLibrary()
  File "/usr/local/lib/python3.10/dist-packages/pynvml.py", line 2250, in _LoadNvmlLibrary
    _nvmlCheckReturn(NVML_ERROR_LIBRARY_NOT_FOUND)
  File "/usr/local/lib/python3.10/dist-packages/pynvml.py", 

In [ ]:
import cudf as pd
from numba import cuda

df = pd.read_csv('/content/gpu_data/MAIN_TABLE_TO_PROCESS.csv', ';')

In [1]:
df_additional_measures = pd.read_csv('/content/gpu_data/ADDITIONAL_MEASURES.csv', ';')
df_additional_tw_measures = pd.read_csv('/content/gpu_data/ADDITIONAL_TOWERCO_MEASURES.csv', ';')
df_exchange_rates = pd.read_csv('/content/gpu_data/EXCHANGE_RATE.csv', ';')
df_general_measures = pd.read_csv('/content/gpu_data/GENERAL_MEASURES.csv', ';')
df_leg_for_lookup_codes = pd.read_csv('/content/gpu_data/LEGEND_FOR_LOOKUP_CODES.csv', ';')

joint_gm_names = ','.join(list(df_general_measures['MEASURE_TYPE'].astype(str).values_host))
joint_gm_values = ','.join(list(df_general_measures['MEASURE_VALUE'].astype(str).values_host))
joint_am_names = ','.join(list(df_additional_measures['MEASURE_TYPE'].astype(str).values_host))
joint_am_values = ','.join(list(df_additional_measures['MEASURE_VALUE'].astype(str).values_host))
joint_tw_names = ','.join(list(df_additional_tw_measures['MEASURE_TYPE'].astype(str).values_host))
joint_tw_values = ','.join(list(df_additional_tw_measures['MEASURE_VALUE'].astype(str).values_host))

df['general_measures_names'] = joint_gm_names
df['general_measures_values'] = joint_gm_values
df['additional_measures_names'] = joint_am_names
df['additional_measures_values'] = joint_am_values
df['additional_tw_measures_names'] = joint_tw_names
df['additional_tw_measures_values'] = joint_tw_values

def replace_formula_usd_kernel(row):
    replaced_formula = row['Formula_USD']

    general_measures_names = row['general_measures_names']
    general_measures_values = row['general_measures_values']
    additional_measures_names = row['additional_measures_names']
    additional_measures_values = row['additional_measures_values']
    additional_tw_measures_names = row['additional_tw_measures_names']
    additional_tw_measures_values = row['additional_tw_measures_values']

    arr_general_measures_names = general_measures_names.split(',')
    arr_general_measures_values = general_measures_values.split(',')
    arr_additional_measures_names = additional_measures_names.split(',')
    arr_additional_measures_values = additional_measures_values.split(',')
    arr_additional_tw_measures_names = additional_tw_measures_names.split(',')
    arr_additional_tw_measures_values = additional_tw_measures_values.split(',')

    for i in range(len(arr_general_measures_names)):
      measure_type = '@' + arr_general_measures_names[i];

      if replaced_formula.contains(measure_type):
        replaced_formula = replaced_formula.replace(measure_type, arr_general_measures_values[i])

    for i in range(len(arr_additional_measures_names)):
      measure_type = '@' + arr_additional_measures_names[i];

      if replaced_formula.contains(measure_type):
        replaced_formula = replaced_formula.replace(measure_type, arr_additional_measures_values[i])

    for i in range(len(arr_additional_tw_measures_names)):
      measure_type = '@' + arr_additional_tw_measures_names[i];

      if replaced_formula.contains(measure_type):
          replaced_formula = replaced_formula.replace(measure_type, arr_additional_tw_measures_values[i])

    return replaced_formula


df['Updated_Formula_USD'] = df.apply(replace_formula_usd_kernel, axis=1)

NameError: name 'pd' is not defined